In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 720 kB 4.2 MB/s 
     |████████████████████████████████| 189 kB 48.3 MB/s 
     |████████████████████████████████| 1.2 MB 43.7 MB/s 
     |████████████████████████████████| 48 kB 4.9 MB/s 
     |████████████████████████████████| 55 kB 4.1 MB/s 
     |████████████████████████████████| 51 kB 144 kB/s 
     |████████████████████████████████| 558 kB 35.3 MB/s 
     |████████████████████████████████| 130 kB 29.2 MB/s 
Mounted at /content/gdrive


In [ ]:
from fastai.vision.all import*
from fastbook import*
matplotlib.rc('image', cmap= 'Greys')

In [ ]:
path = untar_data(URLs.MNIST)

In [ ]:
#PART 1 START HERE. 
Train_8_a = (path/'training'/'8').ls().sorted()
Train_8_b = (path/'training'/'8').ls().sorted()
Train_8_1 = Train_8_a [:int(len(Train_8_a)*0.8)]
Train_8_2 = Train_8_b [:int(len(Train_8_b)*0.8)]



In [ ]:
im8_1_path = Train_8_1[1]
im8_2_path = Train_8_2[1]
im8 = Image.open(im8_1_path)
im8 = Image.open(im8_2_path)

In [ ]:
def matmul(a,b):
    ar,ac = a.shape # n_rows * n_cols
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            for k in range(ac): c[i,j] += a[i,k] * b[k,j]
    return c

In [ ]:
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [ ]:
pred = tensor([2.8,4.9,1.5])
targ = tensor([1.7,7.3,2.6])
error = mse(pred,targ)
error

TensorBase(2.7267)

In [ ]:
#expand_as() and stride() functions 
m = tensor([[1., 2, 3], [4,5,6], [7,8,9]])
c = tensor([10.,20,30])
c.expand_as(m)
t = c.expand_as(m)
t.stride(), t.shape

((0, 1), torch.Size([3, 3]))

In [ ]:
# PART 2 START HERE 
 # Training dataset
threes = (path/'training'/'3').ls().sorted()
sevens = (path/'training'/'7').ls().sorted()
eightees = (path/'training'/'8').ls().sorted()
onees = (path/'training'/'1').ls().sorted()

train1= onees[:int(len(onees)*0.8)]
train3= threes[:int(len(threes)*0.8)]
train7= sevens[:int(len(sevens)*0.8)]
train8= eightees[:int(len(eightees)*0.8)]

threes_tensor = [tensor(Image.open(o)) for o in train3]
onees_tensor = [tensor(Image.open(o)) for o in train1]
eightees_tensor = [tensor(Image.open(o)) for o in train8]
sevens_tensor = [tensor(Image.open(o)) for o in train7]

stacked_sevens = torch.stack(sevens_tensor).float()/255
stacked_threes = torch.stack(threes_tensor).float()/255
stacked_onees = torch.stack(onees_tensor).float()/255
stacked_eightees = torch.stack(eightees_tensor).float()/255

train_x = torch.cat([stacked_threes, stacked_sevens, stacked_onees, stacked_eightees]).view(-1, 28*28)
train_y = tensor([3]*len(train3) + [7]*len(train7) + [8]*len(train8) + [1]*len(train1)).unsqueeze(1)
dset = list(zip(train_x,train_y))
x,y = dset[0]
train_x.shape ,train_y.shape 


(torch.Size([19989, 784]), torch.Size([19989, 1]))

In [ ]:
#Validation dataset 
val1 = onees[int(len(onees)*0.8):] 
val3 = threes[int(len(threes)*0.8):]
val7 = sevens[int(len(sevens)*0.8):]
val8 = eightees[int(len(eightees)*0.8):]

val3_tensor = [tensor(Image.open(o)) for o in val3]
val1_tensor = [tensor(Image.open(o)) for o in val1]
val8_tensor = [tensor(Image.open(o)) for o in val8]
val7_tensor = [tensor(Image.open(o)) for o in val7]

stacked_val7 = torch.stack(val7_tensor).float()/255
stacked_val3 = torch.stack(val3_tensor).float()/255
stacked_val1 = torch.stack(val1_tensor).float()/255
stacked_val8 = torch.stack(val8_tensor).float()/255

valid_x = torch.cat([stacked_val7, stacked_val3, stacked_val1, stacked_val8]).view(-1, 28*28)
valid_y = tensor([3]*len(val3) + [7]*len(val7) + [8]*len(val8) + [1]*len(val1)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))

valid_x.shape, valid_y.shape


(torch.Size([5000, 784]), torch.Size([5000, 1]))

In [ ]:
# testing dataset 
T_3 = (path/'testing'/'3').ls().sorted()
T_7 = (path/'testing'/'7').ls().sorted()
T_8 = (path/'testing'/'8').ls().sorted()
T_1 = (path/'testing'/'1').ls().sorted()

T_1_tensor = [tensor(Image.open(o)) for o in T_1]
T_3_tensor = [tensor(Image.open(o)) for o in T_3]
T_7_tensor = [tensor(Image.open(o)) for o in T_7]
T_8_tensor = [tensor(Image.open(o)) for o in T_8]

TEST_1 = torch.stack(T_1_tensor).float()/255
TEST_3 = torch.stack(T_3_tensor).float()/255
TEST_7 = torch.stack(T_7_tensor).float()/255
TEST_8 = torch.stack(T_8_tensor).float()/255

test_x = torch.cat([TEST_1, TEST_3, TEST_7, TEST_8]).view(-1, 28*28)
test_y = tensor([3]*len(T_3) + [7]*len(T_7) + [8]*len(T_8) + [1]*len(T_1)).unsqueeze(1)
dset = list(zip(test_x,test_y))

test_x.shape, test_y.shape 


(torch.Size([4147, 784]), torch.Size([4147, 1]))

In [ ]:
#initialization 
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

In [ ]:
#weights 
weights = init_params((28*28,2))

In [ ]:
#bias
bias = init_params(1)

In [ ]:
#calculating prediction for all images
def linear1(xb): return xb@weights + bias
preds = linear1(test_x)
preds

tensor([[  0.2837,   1.2217],
        [ -1.1385,   1.9051],
        [ -6.0037,   0.4716],
        ...,
        [ -8.2302,   5.8702],
        [  0.6366,   5.3866],
        [-32.2975,  -5.5150]], grad_fn=<AddBackward0>)

In [ ]:
#checking accuracy set at >0.0 
corrects = (preds>0.0).float() == test_y
corrects




tensor([[False, False],
        [False, False],
        [False, False],
        ...,
        [False,  True],
        [ True,  True],
        [False, False]])

In [ ]:
corrects.float().mean().item()

0.10284543037414551

In [ ]:
preds = linear1(train_x)
((preds>0.0).float() == train_y).float().mean().item()


0.10141411423683167

In [ ]:
def sigmoid(x): return 1/(1+torch.exp(-x))

In [ ]:
# loss function 
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()
     
       

In [ ]:
#re-initialize our parameter 
weights = init_params((28*28,6))
bias = init_params(1)

In [ ]:
# testing batch  
batch = train_x
batch.shape

torch.Size([4147, 784])

In [ ]:
preds = linear1(batch)
preds

tensor([[  6.1591,  -8.3509,  10.8051,  -8.3102,  -3.6773,  -0.0401],
        [ 10.5611,  -7.8121,   8.8161,  -4.3242,  -3.4024,   1.1166],
        [ 14.0621,  -6.7051,  12.4972,  -5.7174,  -3.8254,   1.7105],
        ...,
        [  6.6644,  -9.7997,  -3.1607,  -1.2611,   2.1244, -18.5404],
        [  8.3180, -11.4610,   5.1018,  16.3712,  -7.1321,   2.4677],
        [  5.5884,  -3.8634,   6.4345,  -0.4676,  -3.6123, -13.0101]], grad_fn=<AddBackward0>)

In [ ]:
loss = mnist_loss(preds, train_y)
loss

tensor(0.4436, grad_fn=<MeanBackward0>)

In [ ]:
# gradient function 
def calc_grad(xb, yb, model):
    preds = model(xb)
    loss = mnist_loss(preds, yb)
    loss.backward()

In [ ]:
calc_grad(batch, train_y, linear1)
weights.grad.mean(),bias.grad

(tensor(0.0004), tensor([0.0255]))

In [ ]:
def train_epoch(model, lr, params):
    for xb,yb in dl:
        calc_grad(xb, yb, model)
        for p in params:
            p.data -= p.grad*lr
            p.grad.zero_()

In [ ]:
(preds>0.0).float() == train_y

tensor([[False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        ...,
        [False,  True, False,  True,  True, False],
        [False, False, False, False,  True,  True],
        [False,  True, False,  True,  True, False]])

In [ ]:
# function to calculate our validation accuracy 
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.0) == yb
    return correct.float().mean()

In [ ]:
batch_accuracy(linear1(batch), train_y)

tensor(0.2698)

In [ ]:
def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb,yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)

In [ ]:
validate_epoch(linear1)

0.2869

In [ ]:
lr = 1.
params = weights,bias
train_epoch(linear1, lr, params)
validate_epoch(linear1)

0.2869

In [ ]:
for i in range(20):
    train_epoch(linear1, lr, params)
    print(validate_epoch(linear1), end=' ')

0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 

In [ ]:
linear_model = nn.Linear(28*28,1)

In [ ]:
w,b = linear_model.parameters()
w.shape,b.shape

(torch.Size([1, 784]), torch.Size([1]))

In [ ]:
class BasicOptim:
    def __init__(self,params,lr): self.params,self.lr = list(params),lr

    def step(self, *args, **kwargs):
        for p in self.params: p.data -= p.grad.data * self.lr

    def zero_grad(self, *args, **kwargs):
        for p in self.params: p.grad = None

In [ ]:
#optimizer
opt = BasicOptim(linear_model.parameters(), lr)


In [ ]:
def train_epoch(model):
    for xb,yb in dl:
        calc_grad(xb, yb, model)
        opt.step()
        opt.zero_grad()

In [ ]:
validate_epoch(linear_model)

0.2869

In [ ]:
def train_model(model, epochs):
    for i in range(epochs):
        train_epoch(model)
        print(validate_epoch(model), end=' ')

In [ ]:
linear_model = nn.Linear(28*28,1)
opt = SGD(linear_model.parameters(), lr)
train_model(linear_model, 20)


0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 0.2869 

In [ ]:
dls = DataLoaders(dl, valid_dl)

In [ ]:
learn = Learner(dls, nn.Linear(28*28,1), opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [ ]:
learn.fit(15, lr=lr)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.449812,0.269813,0.269800,00:00
1,0.449812,0.269813,0.269800,00:00
2,0.449812,0.269813,0.269800,00:00
3,0.449812,0.269813,0.269800,00:00
4,0.449812,0.269813,0.269800,00:00
5,0.449812,0.269813,0.269800,00:00
6,0.449812,0.269813,0.269800,00:00
7,0.449812,0.269812,0.269800,00:00
8,0.449811,0.269812,0.269800,00:00
9,0.449811,0.269812,0.269800,00:00


In [ ]:
w1 = init_params((28*28,30))
b1 = init_params(30)
w2 = init_params((30,1))
b2 = init_params(1)

In [ ]:
def simple_net(xb): 
    res = xb@w1 + b1
    res = res.max(tensor(0.0))
    res = res@w2 + b2
    return res

In [ ]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,1)
)

In [ ]:
learn = Learner(dls, simple_net, opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [ ]:
learn.fit(10, 0.1)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.477982,0.274359,0.269800,00:00
1,0.456558,0.271906,0.269800,00:00
2,0.452211,0.271142,0.269800,00:00
3,0.451029,0.270775,0.269800,00:00
4,0.450612,0.270561,0.269800,00:00
5,0.450417,0.270422,0.269800,00:00
6,0.450302,0.270325,0.269800,00:00
7,0.450224,0.270253,0.269800,00:00
8,0.450168,0.270198,0.269800,00:00
9,0.450124,0.270155,0.269800,00:00
